In [1]:
!pip install gensim==4.3.1

  Using cached gensim-4.3.1-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (26.5 MB)
  Using cached smart_open-6.3.0-py3-none-any.whl (56 kB)
  Using cached scipy-1.10.1-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (34.5 MB)
  Using cached numpy-1.24.2-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pathy 0.6.2 requires smart-open<6.0.0,>=5.2.1, but you have smart-open 6.3.0 which is incompatible.
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [2]:
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
import pandas as pd
import numpy as np
from sklearn import svm
import random
from gensim.models import Word2Vec
import gensim.downloader as api
import string
import re
from nltk.tokenize import word_tokenize
from sklearn.impute import SimpleImputer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
nltk.download('vader_lexicon')
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


# iPhone 11

In [3]:
df = pd.read_csv("APPLE_iPhone_SE.csv")

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9713 entries, 0 to 9712
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Ratings  9713 non-null   int64 
 1   Comment  9713 non-null   object
 2   Reviews  9713 non-null   object
dtypes: int64(1), object(2)
memory usage: 227.8+ KB


In [5]:
df

,Ratings,Comment,Reviews
0,5,Super!,Great camera for pics and videos Battery life ...
1,5,Must buy!,Great device. Let me tell the Pros..1. Superb ...
2,5,Great product,"Who all loves older size i.e., 4.7 inch type s..."
3,5,Simply awesome,This iPhone SE is the best phone ever you get....
4,5,Classy product,This is my second iphone after iphone 4s. I’ve...
...,...,...,...
9708,5,Terrific purchase,Absolutely brilliantREAD MORE
9709,5,Classy product,"Superb phone. This is my 4th iPhone, I feel SE..."
9710,5,Awesome,very niceREAD MORE
9711,5,Super!,Loving it as of now. Good Product .READ MORE


In [6]:
df.dropna(inplace=True)

In [7]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    text = re.sub(r'\@\w+', '', text)
    text = re.sub(r'\#\w+', '', text)
    text = re.sub(r'\d+', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = text.strip()
    return text

In [8]:
# tokenization
nltk.download('punkt')
df['clean_text'] = df['Reviews'].apply(clean_text)
df['tokens'] = df['clean_text'].apply(word_tokenize)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [9]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
df['filtered_tokens'] = df['tokens'].apply(lambda x: [word for word in x if word not in stop_words])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
# convert the list of tokens back to text
df['clean_text'] = df['filtered_tokens'].apply(lambda x: ' '.join(x))

## Sentiment Analysis

In [11]:
sid = SentimentIntensityAnalyzer()
df['sentiment'] = df['clean_text'].apply(lambda x: sid.polarity_scores(x)['compound'])

In [12]:
df['sentiment'] = df['sentiment'].apply(lambda x: 'positive' if x > 0 else 'negative')

In [ ]:
def get_sentiment_score(text):
    sentiment_scores = sid.polarity_scores(text)
    return sentiment_scores['compound']

df['sentiment_score'] = df['clean_text'].apply(get_sentiment_score)

In [ ]:
df

In [13]:
#df['Ratings'] = df['Ratings'].astype('float')

In [14]:
positive_threshold = 4.0
negative_threshold = 2.0
positive_text = df[df['Ratings'] >= positive_threshold]
negative_text = df[df['Ratings'] <= negative_threshold]

In [15]:
print('Positive Reviews')
print(positive_text['Reviews'])

Positive Reviews
0       Great camera for pics and videos Battery life ...
1       Great device. Let me tell the Pros..1. Superb ...
2       Who all loves older size i.e., 4.7 inch type s...
3       This iPhone SE is the best phone ever you get....
4       This is my second iphone after iphone 4s. I’ve...
                              ...                        
9708                        Absolutely brilliantREAD MORE
9709    Superb phone. This is my 4th iPhone, I feel SE...
9710                                   very niceREAD MORE
9711         Loving it as of now. Good Product .READ MORE
9712                        NiceElegant ElectricREAD MORE
Name: Reviews, Length: 8505, dtype: object


In [16]:
print('Negative Reviews:')
print(negative_text['Reviews'])

Negative Reviews:
16      Everything is fine of this mobile except batte...
59      It’s not good product ...it’s cost is more qua...
190     Fails to do basic things. I have this phone an...
387     Only good thing about this phone is its proces...
396     It's an outdated very very tiny phone, with a ...
                              ...                        
9567    Battery doesn’t last for a day. I charged it i...
9571    It doesn't matter how the phone performs or ho...
9579                 Its charing cable is faultyREAD MORE
9605    Everything ok, Battery utterly failed. Don’t b...
9618    Heating issues even after latest update and ra...
Name: Reviews, Length: 673, dtype: object


### lowest sentiment text

In [17]:
lowest_sentiment_row = df.sort_values(by='sentiment_score').iloc[0]
print('Review text:', lowest_sentiment_row['Reviews'])
print('Sentiment score:', lowest_sentiment_row['sentiment_score'])

Review text: Low battery conditionREAD MORE
Sentiment score: negative


### highest sentiment text

In [18]:
highest_sentiment_row = df.sort_values('sentiment_score', ascending=False).iloc[0]
print('Review text:', highest_sentiment_row['Reviews'])
print('Sentiment score:', highest_sentiment_row['sentiment_score'])


Review text: Great camera for pics and videos Battery life is good so far with some setting turn of which i never use and when i use i turn those on and i use it in power saving mode all the time so a full day with light gaming of 1hr or more using camera for 1hr or more listening music in my car on youtube and Bluetooth on for couple of hours it gives me full day of battery varing from 4/5 to 6/7 hours sot per dayPerformance is top notch plays every game and every task with easeVery premium phone look...READ MORE
Sentiment score: positive


In [19]:
def clean_text(review_text):
    if isinstance(review_text, str):
        text = review_text.lower() # Convert to lowercase
        tokens = word_tokenize(text) # Tokenize the text
        tokens = [word for word in tokens if word.isalpha()] # Remove punctuation
        tokens = [word for word in tokens if word not in stop_words] # Remove stop words
        return tokens
    else:
        return ""

In [20]:
data =  positive_text + negative_text
labels = ['Positive'] * len(positive_text) + ['Negative']

In [21]:
data = [clean_text(text) for text in data['Reviews']]

vectorizer = TfidfVectorizer()
text_data = vectorizer.fit_transform(df['clean_text'].values.astype('U')) # Convert to Unicode string

labels = df['Reviews'].values

In [22]:

train_data, test_data, train_labels, test_labels = train_test_split(text_data, labels, test_size=0.4, random_state=42)

imputer = SimpleImputer(strategy='mean')
train_data = imputer.fit_transform(train_data)


## Logistic Regression

In [0]:
model = LogisticRegression()
model.fit(train_data, train_labels)

test_predictions = model.predict(test_data)

test_accuracy = accuracy_score(test_labels, test_predictions)
print("Test accuracy:", test_accuracy)

KernelInterrupted: Execution interrupted by the Jupyter kernel.

## Naive Bayes 

In [0]:
model = MultinomialNB()
model.fit(train_data, train_labels)

test_predictions = model.predict(test_data)

test_accuracy = accuracy_score(test_labels, test_predictions)
print("Test accuracy:", test_accuracy)

## Random Forest 

In [0]:
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(train_data, train_labels)

y_pred = model.predict(test_data)

test_accuracy = accuracy_score(test_labels, test_predictions)
print("Test accuracy:", test_accuracy)

# oversampled

In [ ]:
model = LogisticRegression()
model.fit(train_data, train_labels)

test_predictions = model.predict(test_data)

test_accuracy = accuracy_score(test_labels, test_predictions)
print("Test accuracy:", test_accuracy)

In [ ]:
model = MultinomialNB()
model.fit(train_data, train_labels)

test_predictions = model.predict(test_data)

test_accuracy = accuracy_score(test_labels, test_predictions)
print("Test accuracy:", test_accuracy)

In [ ]:
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(train_data, train_labels)

y_pred = model.predict(test_data)

test_accuracy = accuracy_score(test_labels, test_predictions)
print("Test accuracy:", test_accuracy)

In [0]:
selected_word = input('Enter a word to search for: ')
selected_reviews = df[df['clean_text'].str.contains(selected_word)]

sid = SentimentIntensityAnalyzer()
selected_reviews['sentiment_score'] = selected_reviews['Reviews'].apply(lambda x: sid.polarity_scores(x)['compound'])
positive_reviews = selected_reviews[selected_reviews['sentiment_score'] > 0]
negative_reviews = selected_reviews[selected_reviews['sentiment_score'] <= 0]

# Print out the selected reviews by sentiment category
print(f"Positive reviews containing the word '{selected_word}':")
for review_text in positive_reviews['Reviews']:
    print(Reviews)

print(f"\nNegative reviews containing the word '{selected_word}':")
for review_text in negative_reviews['Reviews']:
    print(Reviews)

In [0]:
positive_selected_reviews = selected_reviews[(selected_reviews['sentiment_score'] > 0)]
num_positive_selected = len(positive_selected_reviews[positive_selected_reviews['clean_text'].str.contains(selected_word)])
num_total_selected = len(selected_reviews[selected_reviews['clean_text'].str.contains(selected_word)])
positive_percentage = num_positive_selected / num_total_selected * 100                           

# Print out the percentage of positive reviews containing the selected word
print(f"Percentage of positive reviews containing the word '{selected_word}': {positive_percentage:.2f}%")

In [0]:
negative_selected_reviews = selected_reviews[(selected_reviews['sentiment_score'] <= 0)]
num_negative_selected = len(negative_selected_reviews[negative_selected_reviews['clean_text'].str.contains(selected_word)])
num_total_selected = len(selected_reviews[selected_reviews['clean_text'].str.contains(selected_word)])
negative_percentage = num_negative_selected / num_total_selected * 100                           

# Print out the percentage of negative reviews containing the selected word
print(f"Percentage of negative reviews containing the word '{selected_word}': {negative_percentage:.2f}%")

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=320e99cb-8487-44b0-a3ee-3aa3add03759' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>